In [2]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [7]:
# Create a Fully Connected Network
class NN(nn.Module):
  def __init__(self, in_features, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(in_features, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# model = NN(784, 10)
# random_tensor = torch.randn(64, 784)
# print(model(random_tensor).shape)

In [8]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
# Hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 64
INPUT_SIZE = 784
NUM_CLASSES = 10
EPOCHS = 10

In [13]:
# Load Data
train_data = datasets.MNIST(root='data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
test_data = datasets.MNIST(root='data',
                           train=False,
                           download=True,
                           transform=transforms.ToTensor())

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

train_data, test_data

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: data
     Split: Test
     StandardTransform
 Transform: ToTensor())

In [14]:
# Initialize Network
model = NN(INPUT_SIZE, NUM_CLASSES).to(device)
model

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [15]:
# Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(),
                       lr=LEARNING_RATE)

loss_fn, optimizer

(CrossEntropyLoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ))

In [16]:
# Train Network
model.train()
for epoch in range(EPOCHS):
  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    X = X.reshape(X.shape[0], -1)
    preds = model(X)
    loss = loss_fn(preds, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [17]:
# Check accuracy on training and test data to see how good our model is
def check_accuracy(model, dataloader):

  if dataloader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on testing data")

  num_correct, num_samples = 0, 0

  model.eval()
  with torch.inference_mode():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      X = X.reshape(X.shape[0], -1)
      scores = model(X)
      _, predictions = scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)

  print(f"{num_correct}/{num_samples} correct and the accuracy is {num_correct/num_samples * 100:.2f}")

check_accuracy(model, train_dataloader)
check_accuracy(model, test_dataloader)

Checking accuracy on training data
59094/60000 correct and the accuracy is 98.49
Checking accuracy on testing data
9716/10000 correct and the accuracy is 97.16
